In [3]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [4]:
import sys
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
from typing import Mapping  
import mmh3
from itertools import chain  

In [5]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from nearest_neighbors import SimHash

/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [6]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  
max_n_neighbors = 20
k_values = np.arange(2, max_n_neighbors + 1)


In [7]:
from memory_profiler import profile
%load_ext memory_profiler


In [8]:
%memit x = 10+5

peak memory: 533.75 MiB, increment: 0.00 MiB


In [9]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20
max_n_neighbors = 20
with gzip.open(fasta_gz_file, "rt") as handle:
    flag = 0
    seq_num = 0
    aligned_num = 0
    for record in SeqIO.parse(handle, "fasta"):
        seq_num += 1
        if record.id in max_values.keys():
            aligned_num+=1
            columns = max_values[record.id]
            if int(columns[9])/int(columns[1]) > 0.5:
                flag += 1
    print(flag/seq_num)
    print(aligned_num/seq_num)

0.9493518828919433
0.9829031176667784


In [10]:
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/IGK/pbsim_ONT_93_30k/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/IGK/pbsim_ONT_93_30k//kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/IGK/pbsim_ONT_93_30k/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

In [11]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

(5106, 103266, 9866, 2)

In [12]:
nbr_indices = SimHash().get_neighbors(feature_matrix,read_features,20,tf=True,idf=False)

In [14]:
read_ids = np.array(list(read_features))
graph = OverlapGraph.from_neighbor_indices(
    neighbor_indices=nbr_indices,
    n_neighbors=6,
    read_ids=read_ids,
    require_mutual_neighbors=False,) 

In [ ]:
def remove_small_components(graph, min_component_size=10):
    small_components = set()
    for component in nx.connected_components(graph):
        if len(component) < min_component_size:
            small_components |= component
    graph.remove_nodes_from(small_components)
            
    
def plot_graphs(graphs, reference_graph, metadata, *, min_component_size=10, 
                processes=8, layout_method='stdp', figsize=(6, 6), node_size=3,
    seed: int = 4829, verbose=True):
    axes = []
    new_graphs = []
    g = graphs
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), constrained_layout=True)
    ax1.set_title("All edges")
    ax2.set_title("Correct edges")
    
    g = g.copy()
    remove_small_components(g, min_component_size=min_component_size)
    new_graphs.append(g)
    axes.append(ax1)

    g = g.copy()
    remove_false_edges(g, reference_graph)
    remove_small_components(g, min_component_size=min_component_size)
    new_graphs.append(g)
    axes.append(ax2)

    fig.suptitle('method', ha="center", va="bottom", wrap=True, size=7)

    query_graphs = new_graphs
    def plot(i, pos):
        plot_read_graph(
            ax=axes[i],
            query_graph=query_graphs[i],
            reference_graph=reference_graph,
            metadata=metadata,
            pos=pos,
            node_size=node_size,
        )

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            if layout_method == "umap":
                pos = get_umap_layout(graph=query_graphs[i])
            else:
                pos = get_graphviz_layout(
                    graph=query_graphs[i],
                    figsize=figsize,
                    seed=seed,
                    layout_method=layout_method,
                )
            return i, pos

        def reduce(i, pos):
            if verbose:
                print(i, end=" ")
            plot(i, pos)

        pool.map(work, range(len(query_graphs)), reduce=reduce)
        if verbose:
            print("")

    return fig

In [20]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    all_read_simhash = []
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)

    for read_kmer in read_features.keys():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        conc_hash = np.concatenate(one_read_hash)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)
  
    return reads_simhash_array 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

In [108]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

for repeat in [100]:
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    dfs.append(df)

new = pd.concat(dfs)

80
90
100


In [22]:
df[df.n_neighbors==6 &]

,Unnamed: 0,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
4,4,SimHash,6,20,0.010583,0.000784,0.003138,0.002433,5100,0.998825,1
23,23,SimHash,6,2,0.404492,0.093531,0.090165,0.218224,1299,0.254407,88
42,42,SimHash,6,20,0.732396,0.190486,0.170414,0.463916,200,0.039170,262
61,61,SimHash,6,20,0.733710,0.188656,0.170869,0.459862,212,0.041520,344
80,80,SimHash,6,20,0.732396,0.190486,0.170414,0.463916,200,0.039170,262
...,...,...,...,...,...,...,...,...,...,...,...
1448,156,SimHash,6,60,0.768928,0.201489,0.179972,0.493614,139,0.027223,271
1467,175,SimHash,6,70,0.770857,0.202796,0.180476,0.496959,138,0.027027,270
1486,194,SimHash,6,80,0.775372,0.204339,0.181705,0.501216,136,0.026635,396
1505,213,SimHash,6,90,0.777328,0.204199,0.182141,0.500811,130,0.025460,400


In [12]:
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]

NameError: name 'nbrs' is not defined

In [91]:
for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)


测试hash方法

In [21]:
import hashlib

def simhash(features):
  
  # Generate a hash for each feature
  hashes = [hashlib.sha1(feature).hexdigest() for feature in features]
  
  # Combine the feature hashes to produce the final simhash
  concatenated_hash = ''.join(hashes)
  simhash = hashlib.sha1(concatenated_hash).hexdigest()
  
  return simhash

In [ ]:
x

In [30]:
pip install xxhash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [35]:
import xxhash
x = np.random.rand(1024 * 1024 * 16)
h = xxhash.xxh64()
h.update(x); h.intdigest(); h.reset()

In [36]:
x

array([0.29611066, 0.45210823, 0.21455805, ..., 0.22218214, 0.79713455,
       0.43898445])

In [38]:
## 64位， 40个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:064b}".format(hash_value & 0xFFFFFFFFFFFFFFFF)
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]):  
            kmer_index_str = str(kmer_index)  
            hash_list = _hash(kmer_index_str, seed=s)  
            kmer_hash_indice[kmer_index] = hash_list  
  
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 40, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]


299897765
2378963497
44873935
2670060064
25935175
920060685
1586326711
1550303102
278283145
1103905279
2497529993
993549511
3538826777
827898949
1744767203
1454149367
347405931
293704892
739522939
2787976300
923371613
3028338653
1487664578
1269177166
1108403661
2333147459
3468047501
3767013626
3224645638
216444033
873431322
2428674822
205263707
3132866966
3313916789
964022396
3691436413
768347323
2334284253
3571135595


In [ ]:
rng = np.random.default_rng(seed)
beta = rng.uniform(0, 1, (feature_count, dimension_count))
x = rng.uniform(0, 1, (feature_count, dimension_count))
u1 = rng.uniform(0, 1, (feature_count, dimension_count))
u2 = rng.uniform(0, 1, (feature_count, dimension_count))

for j_sample in range(0, sample_count):
    feature_indices = sparse.find(data[:, j_sample] > 0)[0]
    gamma = -np.log(np.multiply(u1[feature_indices, :], u2[feature_indices, :]))
    t_matrix = np.floor(
        np.divide(
            matlib.repmat(
                np.log(data[feature_indices, j_sample].todense()),
                1,
                dimension_count,
            ),
            gamma,
        )
        + beta[feature_indices, :]
    )
    y_matrix = np.exp(np.multiply(gamma, t_matrix - beta[feature_indices, :]))
    a_matrix = np.divide(
        -np.log(x[feature_indices, :]),
        np.divide(y_matrix, u1[feature_indices, :]),
    )